In [ ]:
# MNQ Range Program 

#by Wolfrank Guzman GitHub: @guzmanwolfrank

In [ ]:
# The goal of this program is to calculate the first 45 min Open Range on MNQ Futures along with the Day Range and Day Change.  

# By creating this dataframe, we can use it to analyze correlations between positive and negative days and their prospective ranges, and the change in these price swings and ranges. 
# We can later apply machine learning predictive models to the data and later backtest, deploy live and see if the correlations and projections hold true. 



In [ ]:
## Data Dictionary

# ABSDayChange:  This column shows the absolute value of the Day's Change which is the Close - Open.  </br>
# DayChange:  The positive or negative value of the Day's change which is the Close - Open. 	</br>
# OpenOnlyRange:  This column shows the value of the min and max value in the first 45 minutes of the Market Open at 9:30.  We are using Equities, not Futures open. 	</br>
# DayRange:  This value contains the distance from the Day's High to Low or vice versa. </br>

In [ ]:
import os
print (os.path.abspath("testfile.ext"))


In [ ]:
import yfinance as yf
import pandas as pd
import warnings 
import seaborn as sns
import matplotlib.pyplot as plt


# Ignore all warnings
warnings.filterwarnings('ignore')

# Set the ticker symbol 
ticker_symbol = "MNQ=F"
# Fetch intraday data using yfinance for the last 60 days with 15-minute intervals
data = yf.download(ticker_symbol, period="60d", interval="15m").round(2)

# Round numbers
data = data.round(2)
#display(data)
# Convert the index to datetime format
data.index = pd.to_datetime(data.index)

# Filter rows for the specific times 09:30 and 10:15
filtered_data = data[(data.index.time == pd.to_datetime("09:30:00").time()) | (data.index.time == pd.to_datetime("10:15:00").time())]
#display(filtered_data)

# Group by date and aggregate the data
combined_data = filtered_data.groupby(filtered_data.index.date).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

# Calculate the range and add it as a new column
combined_data['Range'] = combined_data['High'] - combined_data['Low']

# Round numbers
data = data.round(2)
cdata = combined_data.round(2)
#display(cdata)

# Calculate the average range for different windows of days
windows = [5, 10, 20, 30, 40, 50, 60]
for window in windows:
    avg_range = combined_data['Range'].rolling(window=window).mean().iloc[-1].round(2)
   # print(f"Average Opening Range for last {window} days:", avg_range)

# Calculate the average of the 'Range' column
average_range = combined_data['Range'].mean()

Ddata = yf.download(ticker_symbol, period="60d", interval="1d").round(2)
Ddata['ABSDayChange']= abs(Ddata['Close']- Ddata['Open'])
Ddata['DayChange']= (Ddata['Close']- Ddata['Open'])
Ddata['OpenOnlyRange']= combined_data['Range']
Ddata['DayRange']= Ddata['High']-Ddata['Low']

# Calculate absolute day change
Ddata['ABSDayChange'] = abs(Ddata['Close'] - Ddata['Open'])

# Calculate day change
Ddata['DayChange'] = Ddata['Close'] - Ddata['Open']

# Calculate OpenOnlyRange
Ddata['OpenOnlyRange'] = combined_data['Range']

# Calculate DayRange
Ddata['DayRange'] = Ddata['High'] - Ddata['Low']

# Define the window sizes
windows = [5, 10, 20, 30, 40, 50]


print("Average Open Range:", average_range)


# Calculate rolling averages for each column and window size
for column in ['ABSDayChange', 'OpenOnlyRange', 'DayRange']:
    print(f"Rolling Averages for {column}:")
    for window in windows:
        rolling_avg = Ddata[column].rolling(window=window).mean().iloc[-1]
        print(f" - {window}-day Average: {rolling_avg:.2f}")
    print()
    

display(Ddata)

# Define the window sizes
windows = [5, 10, 20, 30, 40, 50]

# Calculate rolling averages for each column and window size
rolling_avgs = {}
for column in ['ABSDayChange', 'OpenOnlyRange', 'DayRange']:
    rolling_avgs[column] = {}
    for window in windows:
        rolling_avg = Ddata[column].rolling(window=window).mean().iloc[-1]
        rolling_avgs[column][window] = rolling_avg

# Create a DataFrame for rolling averages
rolling_avg_df = pd.DataFrame(rolling_avgs)

# Plot line chart comparing the rolling averages
plt.figure(figsize=(10, 6))
sns.lineplot(data=rolling_avg_df, dashes=False)
plt.title('Rolling Averages Comparison')
plt.xlabel('Window Size (days)')
plt.ylabel('Value')
plt.legend(title='Metric')
plt.xticks(windows)
plt.show()


display(rolling_avg_df)
#New Questions:  What percentage of the day range is the open only range?  

#To answer this question we must make a new column, OpenOnly_as_PCT_of_DayRange
Ddata['OpenOnly_as_PCT_of_DayRange']= Ddata['OpenOnlyRange']/Ddata['DayRange']
PCT_of = Ddata['OpenOnly_as_PCT_of_DayRange']

display(Ddata)

# Count instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 35%
count_greater_than_35 = (Ddata['OpenOnly_as_PCT_of_DayRange'] > 0.35).sum()

# Count instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 50%
count_greater_than_50 = (Ddata['OpenOnly_as_PCT_of_DayRange'] > 0.50).sum()

# Count instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 70%
count_greater_than_70 = (Ddata['OpenOnly_as_PCT_of_DayRange'] > 0.70).sum()

print("Number of instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 35%:", count_greater_than_35)
print("Number of instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 50%:", count_greater_than_50)
print("Number of instances where 'OpenOnly_as_PCT_of_DayRange' is greater than 70%:", count_greater_than_70)


# Calculate the total number of instances in the DataFrame Ddata
total_instances = len(Ddata)

print("Total number of instances in the DataFrame:", total_instances)


In [ ]:
#Findings:  Although not the biggest total price swing, the Open Only Range is responsible for most of the price movement in comparison to the rest of the day accounting for how short the #time frame is.  In other words, the first 45 minutes of Equities Open accounts for 35% or more the day's total movement around 60% of the time!  

#The open is very volatile, and the most dangerous part of the day for traders.  